# Importation:

In [1]:
import pandas as pd
import numpy as np
import re
from unicodedata import normalize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Cleaning:

In [9]:
df = pd.read_csv('indeed.pre_processing.csv')
def salary_man(df):
    salaire_moyen = []
    for i in range(len(df)):
        try:
            salaire_liste = re.findall('(\d+),?',normalize('NFKD',df['salaire'][i]).replace(' ',''))
            mois = re.search('mois',df['salaire'][i])
            jour = re.search('jour',df['salaire'][i])
            heure = re.search('heure',df['salaire'][i])
            if mois:
                if len(salaire_liste) > 1:
                    moy = 12 * (int(salaire_liste[0]) + int(salaire_liste[1])) / 2
                    salaire_moyen.append(moy)
                else:
                    salaire_moyen.append(int(salaire_liste[0]) * 12)
            elif jour:
                # Le nombre de jours travaillés maximum retenu sur la période de référence est de 261 jours.
                if len(salaire_liste) > 1:
                    moy = 261 * (int(salaire_liste[0]) + int(salaire_liste[1])) / 2
                    salaire_moyen.append(moy)
                else:
                    salaire_moyen.append(int(salaire_liste[0]) * 261)
            elif heure:
                # 1600 heures travaillées par an.
                if len(salaire_liste) > 1:
                    moy = (int(salaire_liste[0]) + int(salaire_liste[1])) / 2
                    if moy < 20:
                    #grande chance que le salaire soit exprimé en fait en k euros:
                        salaire_moyen.append(moy * 1600)
                    else:
                        salaire_moyen.append(moy * 1000)
                else:
                    if int(salaire_liste[0]) < 20:
                        salaire_moyen.append(int(salaire_liste[0]) * 1600)
                    else:
                        salaire_moyen.append(int(salaire_liste[0]) * 1000)
            else:
                if len(salaire_liste) > 1:
                    moy = (int(salaire_liste[0]) + int(salaire_liste[1])) / 2
                    salaire_moyen.append(moy)
                else:
                    salaire_moyen.append(int(salaire_liste[0]))
                    
        except:
            salaire_moyen.append('None')
            continue
    return salaire_moyen

salaire_moyen = salary_man(df)

#df.insert(6, "salaire_moyen", salaire_moyen, True)
df['salaire_moyen'] = salaire_moyen

contrat = ['cdi', 'cdd', 'stage', 'alternance', 'freelance']
data = []
for i in range(len(df)):
    inside_data = []
    for ele in contrat:
        pattern = re.compile(r"[\s/\(\),.]"+ele+r"[\s/\(\),.]")
        value = pattern.search(df['description'][i].lower().replace('\n',' ').replace('\r',' '))
        if value:
            inside_data.append(1)
        else:
            inside_data.append(0)
    data.append(inside_data)

desc = titre = [ele+'_desc' for ele in contrat]
contrat_desc = pd.DataFrame(data, columns=desc)

contrat = ['cdi', 'cdd', 'stage', 'alternance', 'freelance']
data = []
for i in range(len(df)):
    inside_data = []
    for ele in contrat:
        pattern = re.compile(r"[\s/\(\),.]"+ele+r"[\s/\(\),.]?")
        value = pattern.search(df['titre'][i].lower().replace('\n',' ').replace('\r',' '))
        if value:
            inside_data.append(1)
        else:
            inside_data.append(0)
    data.append(inside_data)
titre = [ele+'_titre' for ele in contrat]
contrat_titre = pd.DataFrame(data, columns=titre)

liste_langage = ['python', 'r','vba', 'mysql','excel','asp.net','nosql','sql','linux','mongodb',
            'mariadb','java','javascript','php','html','css','sas','c#','ruby','swift','objective-c',
            'vb.net','kotlin','scala','bash','powershell','shell','front end','back end','soap',
            'dasl', 'pict', 'rexx', 'd', 'asp.net', 'f#', 'joy', 'sr', 'back end', 'poplog', 
            'modula-2', 'powershell', 'bash', 'concurrent pascal', 'apl', 'eiffel', 'lisp', 
            'cilk', 'c', 'awk', 'charity', 'bliss', 'spin', 'swift', 'visual basic', 'game maker language',
            'lisaac', 'xl', 'nosql', 'limbo', 'e', 'lava', 'slate', 'revolution', 'coldfusion', 
            'matlab', 'vb.net', 'cobol', 'tcl', 'rpg', 'sas', 'curl', 'front end', 'nemerle', 'f-script', 
            'xml', 's-lang', 'curry', 'basic', 'autoit', 'prolog', 'erlang', 'maya embedded language', 
            'frink', 'oz', 'windows powershell', 'beta', 'moo', 'object-z', 'self', 'leda', 'fortran', 
            'smalltalk', 'afnix', 'mysql', 'm', 'turing', 'jovial', 'lua', 'mondrian', 'io', 'salsa', 
            'kotlin', 'css', 'xotcl', 'perl', 'component pascal', 'mongodb', 'xhtml', 'dibol', 'oberon',
            'ada', 'visual foxpro', 'clist', 'pl/c', 'algol', 'mariadb', 'sql', 'pliant', 'excel', 
            'cobra', 'occam', 'obliq', 'prograph', 'chuck', 'pl/i', 'hypertalk', 'forth', 'ici', 
            'vba', 'beanshell', 'sgml', 'soap', 'opal', 'small', 'pascal', 'applescript', 'roop', 
            'agora', 'moto', 'rapira', 'alf', 'r', 'ml', 'javascript', 'pcastl', 'abcl', 'postscript', 
            'oxygene', 'cecil', 'clean', 'ops5', 'c\\+\\+', 'shell', 'haskell', 'kite', 'cleo', 'q', 'linux', 
                'vbscript', 'joule', 'rebol', 'jass', 'j', 'fril']

liste_outils = ['tableau', 'powerbi','symfony', 'jquery','angular','react','react native','node js','git','github',
            'visual studio','django','flask','api rest','laravel', 'hololens', 'docker', 'jira', 'scrum', 'kanban', 
               'azure','aws', 'teamcity', 'jenkins']

lieu = pd.get_dummies(df['localisation']).drop(['Bordeaux'], axis = 1)
langage = df[list(set(liste_langage))]
outils = df[liste_outils]

X_Y = pd.concat([lieu, langage, outils, contrat_titre, contrat_desc,df['salaire_moyen']],axis = 1)
Y = X_Y['salaire_moyen'][X_Y['salaire_moyen'] != 'None']
X = X_Y[X_Y['salaire_moyen'] != 'None'].drop(['salaire_moyen'], axis = 1)
X_train, X_test, y_trainy, y_testy = train_test_split(X,Y , test_size=0.25, random_state=0)
kmeans = KMeans(n_clusters=4,init = 'k-means++', max_iter=1000).fit(y_trainy.values.reshape(-1, 1))
y_train = kmeans.labels_
y_test = kmeans.predict(y_testy.values.reshape(-1, 1))

# AdaBoost:

## AdaBoost: decision tree et extra tree

In [ ]:
model = AdaBoostClassifier()
hyperparameters = {'base_estimator': [None,
                                      DecisionTreeClassifier(max_depth = 2),
                                      DecisionTreeClassifier(max_depth = 5),
                                      DecisionTreeClassifier(max_depth = 10),
                                      ExtraTreeClassifier(max_depth = 1),
                                      ExtraTreeClassifier(max_depth = 2),
                                      ExtraTreeClassifier(max_depth = 5),
                                      ExtraTreeClassifier(max_depth = 10)
                                      ],                   
                   'n_estimators': [100,150,200,250,300],
                   'learning_rate' : [0.1,0.3],
                   'algorithm' : ['SAMME', 'SAMME.R']
                  }
clf_Ada_tree = GridSearchCV(model, param_grid = hyperparameters, return_train_score=True, error_score='raise')
clf_Ada_tree.fit(X_train, y_train)
result_Ada_tree = pd.DataFrame.from_dict(clf_Ada_tree.cv_results_)
#Predict values based on new parameters
y_pred_Ada_tree = clf_Ada_tree.predict(X_test)
result_Ada_tree.to_csv("result_Ada_tree.csv", index = False)

In [ ]:
print(metrics.accuracy_score(y_test, y_pred_Ada_tree))
metrics.confusion_matrix(y_test, y_pred_Ada_tree)

In [ ]:
plt.scatter(result_Ada_tree['rank_test_score'],result_Ada_tree['mean_test_score'],color='red')
plt.scatter(result_Ada_tree['rank_test_score'],result_Ada_tree['mean_train_score'],color='blue')

In [ ]:
result_Ada_tree[(result_Ada_tree['mean_train_score'] - result_Ada_tree['mean_test_score']) < 0.05].sort_values('rank_test_score')

## AdaBoost: random forest et extra trees

In [ ]:
model = AdaBoostClassifier()
model1 = RandomForestClassifier()
hyperparameters1 = {'n_estimators': [100,150,200],
                   "criterion": ["entropy"],
                   "max_depth": [2, 5, 10],
                   "max_features": ["auto", "log2", None],
                  }
clf = RandomForestClassifier()
base_est_ran = [clf.set_params(**ele) for ele in ParameterGrid(hyperparameters1)]
hyperparameters = {'base_estimator': base_est_ran,                   
                   'n_estimators': [100,150,200,250,300],
                   'learning_rate' : [0.1,0.3],
                   'algorithm' : ['SAMME', 'SAMME.R']
                  }
clf_Ada_ran = GridSearchCV(model, param_grid = hyperparameters, return_train_score=True, error_score='raise')
clf_Ada_ran.fit(X_train, y_train)
result_Ada_ran = pd.DataFrame.from_dict(clf_Ada_ran.cv_results_)
#Predict values based on new parameters
y_pred_Ada_ran = clf_Ada_ran.predict(X_test)
result_Ada_ran.to_csv("result_Ada_ran.csv", index = False)

C:\Users\utilisateur\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
print(metrics.accuracy_score(y_test, y_pred_Ada_ran))
metrics.confusion_matrix(y_test, y_pred_Ada_ran)

In [ ]:
plt.scatter(result_Ada_ran['rank_test_score'],result_Ada_ran['mean_test_score'],color='red')
plt.scatter(result_Ada_ran['rank_test_score'],result_Ada_ran['mean_train_score'],color='blue')

In [ ]:
result_Ada_tree[(result_Ada_ran['mean_train_score'] - result_Ada_ran['mean_test_score']) < 0.05].sort_values('rank_test_score')

In [10]:
model1 = RandomForestClassifier()
hyperparameters1 = {'n_estimators': [100,150,200],
                   "criterion": ["entropy"],
                   "max_depth": [2, 5, 10],
                   "max_features": ["auto", "log2", None],
                  }
clf_RFC = GridSearchCV(model1, param_grid = hyperparameters1, return_train_score=True)

In [13]:
clf_RFC.Parame

{'n_estimators': [100, 150, 200],
 'criterion': ['entropy'],
 'max_depth': [2, 5, 10],
 'max_features': ['auto', 'log2', None]}

In [14]:
from sklearn.model_selection import ParameterGrid
hyperparameters1 = {'n_estimators': [100,150,200],
                   "criterion": ["entropy"],
                   "max_depth": [2, 5, 10],
                   "max_features": ["auto", "log2", None],
                  }
list(ParameterGrid(hyperparameters1))

[{'criterion': 'entropy',
  'max_depth': 2,
  'max_features': 'auto',
  'n_estimators': 100},
 {'criterion': 'entropy',
  'max_depth': 2,
  'max_features': 'auto',
  'n_estimators': 150},
 {'criterion': 'entropy',
  'max_depth': 2,
  'max_features': 'auto',
  'n_estimators': 200},
 {'criterion': 'entropy',
  'max_depth': 2,
  'max_features': 'log2',
  'n_estimators': 100},
 {'criterion': 'entropy',
  'max_depth': 2,
  'max_features': 'log2',
  'n_estimators': 150},
 {'criterion': 'entropy',
  'max_depth': 2,
  'max_features': 'log2',
  'n_estimators': 200},
 {'criterion': 'entropy',
  'max_depth': 2,
  'max_features': None,
  'n_estimators': 100},
 {'criterion': 'entropy',
  'max_depth': 2,
  'max_features': None,
  'n_estimators': 150},
 {'criterion': 'entropy',
  'max_depth': 2,
  'max_features': None,
  'n_estimators': 200},
 {'criterion': 'entropy',
  'max_depth': 5,
  'max_features': 'auto',
  'n_estimators': 100},
 {'criterion': 'entropy',
  'max_depth': 5,
  'max_features': 'aut

In [29]:
ParameterGrid(hyperparameters1)[0]

{'n_estimators': 100,
 'max_features': 'auto',
 'max_depth': 2,
 'criterion': 'entropy'}

In [32]:
clf = RandomForestClassifier()
hyperparameter = [{clf.set_params(**ele)} for ele in ParameterGrid(hyperparameters1)]
hyperparameter

[{RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
              max_depth=10, max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False)},
 {RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
              max_depth=10, max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False)},
 {RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
              max_depth=10, max_feature

In [27]:
bob = [{i} for i in range(10)]
bob

[{0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}]

In [24]:
GridSearchCV(model1, param_grid = hyperparameters1, return_train_score=True).param_distri

{'n_estimators': [100, 150, 200],
 'criterion': ['entropy'],
 'max_depth': [2, 5, 10],
 'max_features': ['auto', 'log2', None]}

In [21]:
help(GridSearchCV)

Help on class GridSearchCV in module sklearn.model_selection._search:

class GridSearchCV(BaseSearchCV)
 |  GridSearchCV(estimator, param_grid, scoring=None, fit_params=None, n_jobs=None, iid='warn', refit=True, cv='warn', verbose=0, pre_dispatch='2*n_jobs', error_score='raise-deprecating', return_train_score='warn')
 |  
 |  Exhaustive search over specified parameter values for an estimator.
 |  
 |  Important members are fit, predict.
 |  
 |  GridSearchCV implements a "fit" and a "score" method.
 |  It also implements "predict", "predict_proba", "decision_function",
 |  "transform" and "inverse_transform" if they are implemented in the
 |  estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated grid-search over a parameter grid.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object.
 |      This is assumed to implement the scikit-learn estimator 

In [23]:
RandomForestClassifier(list(ParameterGrid(hyperparameters1)))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0,
            n_estimators=[{'criterion': 'entropy', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 100}, {'criterion': 'entropy', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 150}, {'criterion': 'entropy', 'max_depth': 2, 'max_features': 'auto', 'n_estimators': 200}, {'criterion': 'entropy', ...mators': 150}, {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'n_estimators': 200}],
            n_jobs=None, oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
centre = [1 if (df['localisation'][i] in df['adresse'][i]) else 0 for i in range(len(df))]
len(centre)
df['centre'] = centre
df['centre']